# OR 2-2, something with football i guess

In [1]:
import numpy as np
import pandas as pd

import os

In [2]:
data_files = os.listdir("data")
data = pd.read_csv(os.path.join("data", data_files[0]))

for data_file in data_files[1:]:
    data = pd.concat([data, pd.read_csv(os.path.join("data", data_file))])

# Remove whitespaces
data.loc[:, "HomeTeam"] = data["HomeTeam"].str.strip()
data.loc[:, "AwayTeam"] = data["AwayTeam"].str.strip()
data

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Unnamed: 58,Unnamed: 59,Unnamed: 60
0,N1,29/08/08,Vitesse,Groningen,0.0,4.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,N1,30/08/08,Nijmegen,Graafschap,2.0,0.0,H,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,N1,30/08/08,Roda JC,Twente,1.0,1.0,D,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N1,30/08/08,Utrecht,PSV Eindhoven,1.0,5.0,A,0.0,3.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N1,30/08/08,Willem II,Ajax,2.0,1.0,H,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,N1,06/05/12,Heracles,Nijmegen,1.0,2.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302,N1,06/05/12,NAC Breda,Waalwijk,3.0,2.0,H,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,N1,06/05/12,Roda,Utrecht,1.0,3.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,N1,06/05/12,Vitesse,Ajax,1.0,3.0,A,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
scores = data.loc[:, ["HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "Date"]].set_index(["HomeTeam", "AwayTeam"]).rename({"FTHG": "HomeGoals", "FTAG": "AwayGoals", "FTR": "HomeWin"}, axis=1)
scores["HomeWin"] = scores["HomeWin"] == "H"
scores = scores.dropna()
new_index = list(scores.index)

for i, v in enumerate(scores.iterrows()):
    clubs = v[0]
    res = v[1]
    if clubs[0] > clubs[1]:
        new_index[i] = (clubs[1], clubs[0])
        scores.iloc[i] = (res["AwayGoals"], res["HomeGoals"], not res["HomeWin"], res["Date"])

scores.index = pd.MultiIndex.from_tuples(new_index)
scores = scores.rename({"HomeGoals": "FirstGoals", "AwayGoals": "SecondGoals", "HomeWin": "FirstWin"}, axis=1)
scores

,,FirstGoals,SecondGoals,FirstWin,Date
Groningen,Vitesse,4.0,0.0,True,29/08/08
Graafschap,Nijmegen,0.0,2.0,False,30/08/08
Roda JC,Twente,1.0,1.0,False,30/08/08
PSV Eindhoven,Utrecht,5.0,1.0,True,30/08/08
Ajax,Willem II,1.0,2.0,False,30/08/08
...,...,...,...,...,...
Heracles,Nijmegen,1.0,2.0,False,06/05/12
NAC Breda,Waalwijk,3.0,2.0,True,06/05/12
Roda,Utrecht,1.0,3.0,False,06/05/12
Ajax,Vitesse,3.0,1.0,True,06/05/12


We can define the weights as follows:
$f_{i, j} = \frac{l_{i, j}}{2g_{i, j}} + \frac{c_{i, j}}{2c_{i, j} + 2s_{i, j}}$,
where $l_{i, j}$ is games lost, $g_{i, j}$  is total games. $c_{i, j}$ is the total goals conceded, $s_{i, j}$ is total goals scored. So first we need to get all those statistics. Note: $f_{i, j} + f_{j, i} = 1$.

In [4]:
stats = scores.groupby(scores.index).sum()
number_of_games = scores.groupby(scores.index).size().to_frame("GamesNumber")
stats = stats.merge(number_of_games, left_index=True, right_index=True)
stats.index = pd.MultiIndex.from_tuples(stats.index)
stats

FirstGoals  SecondGoals  FirstWin  GamesNumber
AZ Alkmaar Ajax             30.0         56.0        11           27
           Cambuur          14.0          5.0         7            8
           Den Haag         58.0         23.0        20           26
           Dordrecht         5.0          1.0         2            2
           Excelsior        25.0         21.0         8           14
...                          ...          ...       ...          ...
Vitesse    Zwolle           40.0         20.0        14           20
Volendam   Willem II         4.0          3.0         1            2
Waalwijk   Willem II         5.0         13.0         0            9
           Zwolle            8.0         13.0         4            9
Willem II  Zwolle           16.0         21.0         8           18

[360 rows x 4 columns]

In [6]:
weights_series = (stats["GamesNumber"] - stats["FirstWin"])/(2 * stats["GamesNumber"]) + (stats["SecondGoals"]/(2*stats["FirstGoals"] + 2*stats["SecondGoals"]))

weights_copy = weights_series.copy()
second_index = pd.MultiIndex.from_arrays([weights_copy.index.get_level_values(1), weights_copy.index.get_level_values(0)])
weights_copy.index = second_index

weights = pd.concat([weights_series, weights_copy]).to_frame().sort_index().unstack()
weights

0                                                    \
                 AZ Alkmaar      Ajax   Cambuur  Den Haag Dordrecht Excelsior   
AZ Alkmaar              NaN  0.621878  0.194079  0.257360  0.083333  0.442547   
Ajax               0.621878       NaN  0.154762  0.175689  0.392857  0.103774   
Cambuur            0.194079  0.154762       NaN  0.633333  0.100000  0.477273   
Den Haag           0.257360  0.175689  0.633333       NaN  0.000000  0.323892   
Dordrecht          0.083333  0.392857  0.100000  0.000000       NaN  0.166667   
Excelsior          0.442547  0.103774  0.477273  0.323892  0.166667       NaN   
FC Emmen           0.033333  0.038462       NaN  0.746154       NaN  0.166667   
Feyenoord          0.571594  0.235151  0.888889  0.612727  0.900000  0.748626   
For Sittard        0.134615  0.048387  1.000000  0.650000       NaN  0.857143   
Go Ahead Eagles    0.232143  0.119048  0.115385  0.583333  0.166667  0.550000   
Graafschap         0.146774  0.058140  0.527778  0.627778       NaN  0.598214   
Groningen          0.430556  0.221625  0.514881  0.645970  0.875000  0.635714   
Heerenveen         0.449495  0.203384  0.560714  0.610723  0.416667  0.644377   
Heracles           0.211829  0.211794  0.397727  0.635063  0.535714  0.538462   
NAC Breda          0.383333  0.102431  0.900000  0.443376  0.550000  0.560870   
Nijmegen           0.454806  0.181092  0.305882  0.431402       NaN  0.587500   
PSV Eindhoven      0.638596  0.421703  0.842365  0.883682  0.928571  0.853175   
Roda               0.436508  0.189610  0.583333  0.479592       NaN  0.729310   
Roda JC            0.090909  0.125000       NaN  0.708333       NaN       NaN   
Sparta             0.000000  0.464286       NaN  0.483333       NaN       NaN   
Sparta Rotterdam   0.298611  0.106061  0.333333  0.638587       NaN  0.330882   
Twente             0.445000  0.227436  0.761905  0.610897  1.000000  0.541667   
Utrecht            0.453661  0.423521  0.411638  0.554945  0.909091  0.642857   
VVV Venlo          0.240000  0.040984       NaN  0.461594       NaN  0.420833   
Vitesse            0.370445  0.299451  0.723810  0.533582  0.909091  0.588040   
Volendam           0.000000  0.166667       NaN  0.000000       NaN       NaN   
Waalwijk           0.229968  0.080128  0.200000  0.453704       NaN  0.500000   
Willem II          0.365090  0.161603  0.585714  0.307453  0.928571  0.387255   
Zwolle             0.234893  0.124269  0.461694  0.656513  0.928571  0.738889   

                                                                  ...  \
                  FC Emmen Feyenoord For Sittard Go Ahead Eagles  ...   
AZ Alkmaar        0.033333  0.571594    0.134615        0.232143  ...   
Ajax              0.038462  0.235151    0.048387        0.119048  ...   
Cambuur                NaN  0.888889    1.000000        0.115385  ...   
Den Haag          0.746154  0.612727    0.650000        0.583333  ...   
Dordrecht              NaN  0.900000         NaN        0.166667  ...   
Excelsior         0.166667  0.748626    0.857143        0.550000  ...   
FC Emmen               NaN  0.776667    0.488095             NaN  ...   
Feyenoord         0.776667       NaN    0.309524        0.222857  ...   
For Sittard       0.488095  0.309524         NaN        0.785714  ...   
Go Ahead Eagles        NaN  0.222857    0.785714             NaN  ...   
Graafschap        0.833333  0.305172    0.550000             NaN  ...   
Groningen         0.596154  0.291855    0.723810        0.757500  ...   
Heerenveen        0.314286  0.412306    0.519231        0.690476  ...   
Heracles          0.645833  0.277778    0.690476        0.600962  ...   
NAC Breda         0.125000  0.274573    0.187500        0.568182  ...   
Nijmegen               NaN  0.344633    0.875000        0.450000  ...   
PSV Eindhoven     0.882353  0.444009    0.937500        0.923077  ...   
Roda                   NaN  0.238095         NaN        0.416667  ...   
Roda JC                NaN  0.256579         NaN      